# Assignment 5 - Population Analyis

### Authored by: Stephen Kerr

This assingment is broken down into 3 parts.
1. Differences between the sexes by age in Ireland,
2. Grouped population within 5 years of a given varaible age, then look at the differences between them,
3. Regional Differences between the sexes to identify which region has be biggest difference for a given age group,

This Notebook will be divided into 3 Parts to address each assessment part seperately. 

The Population data analysis was gathered from the Irish CSO from the 2022 Census.  
See their website [here](https://www.cso.ie/en/index.html).

## Environemnt Setup

In [1]:
# Imports 

import pandas as pd 

## Load the Data

In [2]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail()
df.to_csv("raw_population_data.csv")

## Cleaning the data for analysis

In [3]:
# Checking the headers of the dataframe

headers = df.columns.to_list()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [4]:
# Dropping unnecessary columns from the dataframe

drop_headers_list = ['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'C02076V03371',
 'C03789V04537',
 'UNIT',
]
df.drop(columns=drop_headers_list, inplace=True)
df.head()

,Sex,Single Year of Age,Administrative Counties,VALUE
0,Both sexes,All ages,Ireland,5149139
1,Both sexes,All ages,Carlow County Council,61968
2,Both sexes,All ages,Dublin City Council,592713
3,Both sexes,All ages,Dún Laoghaire Rathdown County Council,233860
4,Both sexes,All ages,Fingal County Council,330506


In [5]:
# # Remove 'Both Sexes' from the dataframe for clearer analysis
df = df[df['Sex'] != 'Both sexes']

In [6]:
# Cleaning up the 'Single Age' column

df = df[df["Single Year of Age"] != "All ages"] # Remove the 'All ages' rows
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0') # Replace 'Under 1 year' with '0'
df['Single Year of Age'] = df['Single Year of Age'].str.replace(r'\D', '', regex=True) # Remove any non-digit characters using regex
# df['Single Year of Age'].unique() # Check unique values after cleaning

In [7]:
# Converting 'Single Year of Age' to integer type

# df.info() # checking the data types and non-null counts
df['Single Year of Age'] = df['Single Year of Age'].astype('Int64') # Convert 'Single Year of Age' to integer
df.info() # checking the data types and non-null counts after conversion

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   Int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 258.8+ KB


In [ ]:
# Creating a Pivot Table to analyze population by Age, Sex and Administrative Counties

# added .astype(int) to convert float to int as I was getting float values 
df_analysis = pd.pivot_table(df, 
                            values='VALUE', 
                            index="Single Year of Age", 
                            columns=["Administrative Counties", "Sex"]
                            ).astype(int) 
# df_analysis.info()
df_analysis.to_csv("population_by_age_and_county.csv")

In [9]:
df_analysis.head()

Administrative Counties Carlow County Council      Cavan County Council       \
Sex                                    Female Male               Female Male   
Single Year of Age                                                             
0                                         353  346                  501  505   
1                                         302  347                  477  527   
2                                         334  355                  520  541   
3                                         378  376                  525  560   
4                                         369  376                  492  555   

Administrative Counties Clare County Council      Cork City Council        \
Sex                                   Female Male            Female  Male   
Single Year of Age                                                          
0                                        691  686              1124  1159   
1                                        704  656              1136  1148   
2                                        744  729              1162  1144   
3                                        745  723              1106  1188   
4                                        766  739              1101  1142   

Administrative Counties Cork County Council        ...  \
Sex                                  Female  Male  ...   
Single Year of Age                                 ...   
0                                      2055  2135  ...   
1                                      2045  2070  ...   
2                                      2106  2184  ...   
3                                      2177  2260  ...   
4                                      2262  2358  ...   

Administrative Counties Tipperary County Council        \
Sex                                       Female  Male   
Single Year of Age                                       
0                                            915  1017   
1                                            872   936   
2                                            879   957   
3                                           1019  1029   
4                                            998  1074   

Administrative Counties Waterford City & County Council       \
Sex                                              Female Male   
Single Year of Age                                             
0                                                   662  678   
1                                                   602  664   
2                                                   665  664   
3                                                   721  755   
4                                                   634  756   

Administrative Counties Westmeath County Council      Wexford County Council  \
Sex                                       Female Male                 Female   
Single Year of Age                                                             
0                                            535  594                    882   
1                                            602  556                    878   
2                                            532  576                    921   
3                                            558  590                    903   
4                                            609  599                    921   

Administrative Counties       Wicklow County Council        
Sex                      Male                 Female  Male  
Single Year of Age                                          
0                         910                    800   910  
1                         846                    820   862  
2                         914                    890   906  
3                         961                    886   976  
4                        1020                    887  1010  

[5 rows x 64 columns]

## Part 1

Brief: 
> Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.  
> Weighted mean age (by sex)  
> The difference between the sexes by age  
>This part does not need to look at the regions.  